In [1]:
import os
import h5py
import numpy as np
from IPython.lib.deepreload import reload
import matplotlib.pyplot as plt
from collections import OrderedDict
import skimage.io as io
from tqdm import tqdm
import time as ttime

%matplotlib qt

In [2]:
import PyQt5

In [ ]:
from xrdmaptools.utilities.math import tth_2_d, d_2_tth, energy_2_wavelength, tth_2_q, q_2_tth
from xrdmaptools.reflections.SpotModels import GaussianFunctions, LorentzianFunctions
from itertools import permutations
from itertools import product



# Re-write for the crystal class...
def metric_tensor(a, b, c, alpha, beta, gamma):
    A = a**2
    B = b**2
    C = c**2
    D = 2 * b * c * np.cos(alpha)
    E = 2 * a * c * np.cos(beta)
    F = 2 * a * b * np.cos(gamma)
    return (A, B, C, D, E, F)

def S_tensor(a, b, c, alpha, beta, gamma):
    V = a * b * c * np.sqrt(1
                            - np.cos(alpha)**2
                            - np.cos(beta)**2
                            - np.cos(gamma)**2
                            - 2 * np.cos(alpha) * np.cos(beta) * np.cos(gamma))
    S11 = b**2 * c**2 * np.sin(alpha)**2
    S22 = a**2 * c**2 * np.sin(beta)**2
    S33 = a**2 * b**2 * np.sin(gamma)**2
    S12 = a * b * c**2 * (np.cos(alpha) * np.cos(beta) - np.cos(gamma))
    S23 = a**2 * b * c * (np.cos(beta) * np.cos(gamma) - np.cos(alpha))
    S13 = a * b**2 * c * (np.cos(gamma) * np.cos(alpha) - np.cos(beta))

    return (V, S11, S22, S33, S12, S23, S13)


def hkl_2_d(hkls, a, b, c, alpha, beta, gamma):
    VS = np.round(S_tensor(a, b, c, alpha, beta, gamma), 8)

    d_list = []
    for hkl in hkls:
        h, k, l = hkl
        d = VS[0] * (VS[1] * (h**2)
                     + VS[2] * (k**2)
                     + VS[3] * (l**2)
                     + VS[4] * (h * k) * 2
                     + VS[5] * (k * l) * 2
                     + VS[6] * (h * l * 2)
                     )**(-0.5)
        
        d_list.append(d)
    
    return np.array(d_list)


def d_spacings(q_range, a, b, c, alpha, beta, gamma):
    q_min, q_max = q_range
    
    possible_hkls = list(product(range(10), repeat=3))[1:]

    VS = np.round(S_tensor(a, b, c, alpha, beta, gamma), 8)

    d_list = []
    for hkl in possible_hkls:
        h, k, l = hkl
        d = VS[0] * (VS[1] * (h**2)
                     + VS[2] * (k**2)
                     + VS[3] * (l**2)
                     + VS[4] * (h * k) * 2
                     + VS[5] * (k * l) * 2
                     + VS[6] * (h * l * 2)
                     )**(-0.5)
        
        if 2 * np.pi / d < q_min:
            continue
        elif 2 * np.pi / d > q_max:
            continue
        else:
            d_list.append(d)
    
    d_list = np.unique(np.round(d_list, 8))
        
    return sorted(d_list, reverse=True)


def pattern_estimate_guess(tth, intensity, a, b, c, alpha, beta, gamma, wavelength=test.wavelength, window=20):
    q = tth_2_q(tth, wavelength=wavelength)
    q[np.isnan(q)] = 0
    q_range = [np.min(q), np.max(q)]

    d = d_spacings(q_range, a, b, c, alpha, beta, gamma)
    q_cen = 2 * np.pi / np.array(d)

    args = []
    for i in range(len(q_cen)):
        cen_idx = np.argmin(np.abs(q - q_cen[i]))
        amp = intensity[cen_idx]
        q_vals = q[cen_idx - window : cen_idx + window]
        int_vals = intensity[cen_idx - window : cen_idx + window]
        std_tth = np.sqrt(np.cov(q_vals, aweights=int_vals))
        fwhm = std_tth * 2 * np.sqrt(2 * np.log(2))

        args.extend([
            amp,
            fwhm
        ]) 
    
    return args


def pattern_estimate(tth, *args, wavelength=test.wavelength):
    a, b, c, alpha, beta, gamma = args[:6]
    int_fwhm = args[6:]
    
    q = tth_2_q(tth, wavelength=wavelength)
    q[np.isnan(q)] = 0
    q_range = [np.min(q), np.max(q)]

    d = d_spacings(q_range, a, b, c, alpha, beta, gamma)
    q_cen = 2 * np.pi / np.array(d)

    args = [0]
    for i in range(len(q_cen)):
        args.extend([
            int_fwhm[::2][i],
            q_cen[i],
            int_fwhm[1::2][i]
        ])

    intensity = LorentzianFunctions.multi_1d(q, *args)

    return intensity


def pattern_estimate_from_phase_guess(tth, intensity, window=20):
    phases = [test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']]

    q = tth_2_q(tth, wavelength=test.wavelength)
    q[np.isnan(q)] = 0

    d = []
    for phase in phases:
        lattice_params = list(phase.lattice._parameters.values())
        lattice_params[3:] = np.radians(lattice_params[3:])

        d.extend(hkl_2_d(phase.reflections['hkl'],
                         *lattice_params))

    q_cen = 2 * np.pi / np.array(d)

    args = [np.min(intensity[20:-20])]
    for i in range(len(q_cen)):
        cen_idx = np.argmin(np.abs(q - q_cen[i]))
        amp = intensity[cen_idx] - np.min(intensity)
        
        q_vals = q[cen_idx - window : cen_idx + window]
        int_vals = intensity[cen_idx - window : cen_idx + window]
        int_vals[int_vals < 0] = 0
        
        std_tth = np.sqrt(np.cov(q_vals, aweights=int_vals))
        fwhm = std_tth * 2 * np.sqrt(2 * np.log(2))

        args.extend([
            amp,
            fwhm
        ])

    return args


def pattern_estimate_from_phase(tth, *args):
    phases = [test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']]
    lattice_params = np.array(args[:6 * len(phases)]).reshape(-1, 6)
    offset = args[6 * len(phases)]
    int_fwhm = args[6 * len(phases) + 1:]

    q = tth_2_q(tth, wavelength=test.wavelength)
    q[np.isnan(q)] = 0

    d = []
    for i, phase in enumerate(phases):
        d.extend(hkl_2_d(phase.reflections['hkl'],
                         *lattice_params[i]))
    
    q_cen = 2 * np.pi / np.array(d)

    args = [offset]
    for i in range(len(q_cen)):
        args.extend([
            int_fwhm[::2][i],
            q_cen[i],
            int_fwhm[1::2][i]
        ])

    intensity = LorentzianFunctions.multi_1d(q, *args)

    return intensity

def generate_bounds(phases, args):

    lattice_params = np.array(args[:6 * len(phases)]).reshape(-1, 6)
    offset = args[6 * len(phases)]
    int_fwhm = args[6 * len(phases) + 1:]

    low_bounds, upr_bounds = [], []

    for lattice_params_i in lattice_params:
        a, b, c, alpha, beta, gamma = lattice_params_i

        for const in [a, b, c]:
            # Ridiculous bounds of 5% strain
            low_bounds.append(0.95 * const)
            upr_bounds.append(1.05 * const)
        
        for angle in [alpha, beta, gamma]:
            low_bounds.append(angle - np.radians(5))
            upr_bounds.append(angle + np.radians(5))

    # offset bound
    low_bounds.append(0)
    upr_bounds.append(5 * offset)

    # amp and fwhm
    for i, val in enumerate(int_fwhm):
        if i % 2 == 0: # even, amp
            low_bounds.append(0)
            upr_bounds.append(3 * val)
        else: # odd, fwhm
            low_bounds.append(0.001)
            upr_bounds.append(3 * val)

    return [low_bounds, upr_bounds]


In [ ]:
map_indices = (8, 31)

lattice_params = [3.53, 3.52, 3.529, np.radians(90), np.radians(90), np.radians(90)]

lattice_params = []
for phase in [test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']]:
    lattice_params_i = list(phase.lattice._parameters.values())
    lattice_params_i[3:] = np.radians(lattice_params_i[3:])
    lattice_params.extend(lattice_params_i)
#p0 = pattern_estimate_guess(test.tth, test.map.integrations[map_indices], *lattice_params, wavelength=test.wavelength)
#p_guess = lattice_params + p0

p0 = pattern_estimate_from_phase_guess(test.tth, test.map.integrations[map_indices])
p_guess = lattice_params + p0

intensity = pattern_estimate_from_phase(test.tth, *p_guess)

bounds = generate_bounds([test.phases['304SS-austenite'],
              test.phases['platinum'],
              test.phases['tungsten_2']], p_guess)

In [ ]:
from scipy.optimize import curve_fit
from xrdmaptools.utilities.math import compute_r_squared

#popt, pcov = curve_fit(pattern_estimate, test.tth, test.map.integrations[map_indices], p0=p_guess)
popt, pcov = curve_fit(pattern_estimate_from_phase, test.tth, test.map.integrations[map_indices], p0=p_guess, maxfev=1000, bounds=bounds)
r_squared = compute_r_squared(test.map.integrations[map_indices], pattern_estimate_from_phase(test.tth, *popt))
print(r_squared)
fit_int = pattern_estimate_from_phase(test.tth, *popt)

0.7831190731325022


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(test.tth, test.map.integrations[map_indices], c='k', label='data')
ax.plot(test.tth, intensity, '--', c='k', alpha=0.5, label='guess', lw=1)
ax.plot(test.tth, fit_int, '--', c='r', label='fit')
ax.plot(test.tth, test.map.integrations[map_indices] - fit_int - 0.1, c='gray', label='residual')

ax.legend()

fig.show()

In [ ]:
test.plot_interactive_integration_map()

In [ ]:
from sklearn.decomposition import PCA

n_components = 12

#X = xrfmap.copy().T
X = rescale_array(xrfmap.copy().T, arr_min=0, upper=100)
X = test.map.integrations.copy().T
#X = np.vstack([rescale_array(xrfmap.copy().T, arr_min=0, upper=100), test.map.integrations.copy().T])

h, w = X.shape[1], X.shape[2]

print("Extracting the top %d eigenpatterns from %d patterns"
      % (n_components, X.shape[0]))
t0 = ttime.time()
pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(X.reshape(len(X), h * w))

x_pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit_transform(X.reshape(len(X), h * w))

eigen = pca.components_.reshape((n_components, h, w))

print("done in %0.3fs" % (ttime.time() - t0))

Extracting the top 12 eigenpatterns from 3086 patterns
done in 0.359s


In [ ]:
from sklearn.decomposition import NMF

n_components = 6

#X = xrfmap.copy().T
X = rescale_array(xrfmap.copy().T, arr_min=0, upper=100)
#X = test.map.integrations.copy().T
#X = np.vstack([rescale_array(xrfmap.copy().T, arr_min=0, upper=100), test.map.integrations.copy().T])
X[X < 0] = 0

h, w = X.shape[1], X.shape[2]

print("Extracting the top %d eigenpatterns from %d patterns"
      % (n_components, X.shape[0]))
t0 = ttime.time()
pca = NMF(n_components=n_components).fit(X.reshape(len(X), h * w))

x_pca = NMF(n_components=n_components).fit_transform(X.reshape(len(X), h * w))

eigen = pca.components_.reshape((n_components, h, w))

print("done in %0.3fs" % (ttime.time() - t0))

Extracting the top 6 eigenpatterns from 4096 patterns


c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


done in 3.131s


c:\Users\emusterma\AppData\Local\anaconda3\envs\sXRDMap-py311-1.0\Lib\site-packages\sklearn\decomposition\_nmf.py:1770: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(5, 5), dpi=200)


for i, axi in enumerate(axes):

    axi[0].imshow(eigen[i])
    axi[0].set_xticks([])
    axi[0].set_yticks([])

    #axi[1].plot(xrf_energy, x_pca[:, i])
    #axi[1].plot(test.tth, x_pca[:, i])
    plot_plot = x_pca[:, i]
    axi[1].plot(xrf_energy, plot_plot)
    #axi[1].plot(test.tth, plot_plot)
    axi[1].set_ylim(np.min(plot_plot), np.max(plot_plot))
    axi[1].set_yticks([])

fig.show()

In [ ]:
len(test.tth)

3086

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(5, 5), dpi=200)


for i, axi in enumerate(axes):

    axi[0].imshow(eigen[i])
    axi[0].set_xticks([])
    axi[0].set_yticks([])

    #axi[1].plot(xrf_energy, x_pca[:, i])
    #axi[1].plot(test.tth, x_pca[:, i])
    plot_plot = x_pca[:len(xrf_energy), i]
    axi[1].plot(xrf_energy, plot_plot)
    axi[1].set_ylim(np.min(plot_plot), np.max(plot_plot))
    axi[1].set_yticks([])

    #axi[2].plot(xrf_energy, x_pca[:, i])
    #axi[2].plot(test.tth, x_pca[:, i])
    plot_plot = x_pca[len(xrf_energy):, i]
    axi[2].plot(test.tth, plot_plot)
    axi[2].set_ylim(np.min(plot_plot), np.max(plot_plot))
    axi[2].set_yticks([])

fig.show()

In [ ]:
test.map_extent()

[-10.203479, 9.687759, -20.038216, 20.00252]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

xrf_cen = 2700
x_signal = rescale_array(np.sum(xrfmap[:, :, xrf_cen - 15:xrf_cen + 15], axis=(-1)))
x_energy = np.mean(xrf_energy[xrf_cen - 15:xrf_cen + 15])

tth_cen = 830
y_signal = rescale_array(np.sum(test.map.integrations[:, :, tth_cen - 30:tth_cen + 30], axis=(-1)))
y_tth = np.mean(test.tth[tth_cen - 30:tth_cen + 30])

ax.scatter(x_signal, y_signal, s=1, alpha=0.5, c='k')
ax.set_xlabel(f'XRF Energy around {x_energy:.2f} keV')
ax.set_ylabel(f'XRD 2θ around {y_tth:.2f}°')

corr_signal = x_signal * y_signal

fig.show()

test.plot_map(corr_signal)

In [ ]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
#test.load_phase('AMCSD\\Hematite_0017806.cif', filedir=cif_dir, phase_name="hematite")
#test.load_phase('AMCSD\\Iron-alpha_0011214.cif', filedir=cif_dir, phase_name="iron-ferrite")
#test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="iron-austenite")\

test.load_phase('Unknown\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 new")

test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="platinum")
test.load_phase('AMCSD\\Stibnite_0008636.cif', filedir=cif_dir, phase_name="stibnite")
test.load_phase('AMCSD\\Tungsten_0011236.cif', filedir=cif_dir, phase_name="tungsten_2")
test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="304SS-austenite")
test.phases['304SS-austenite'].a = 3.529

In [ ]:
[phase.get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)),
                            ignore_less=0)
    for phase in test.phases.values()];

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array


def generate_reciprocal_lattice(phase, qmax):
    # Example
    # all_hkls, all_qs, all_fs = generate_reciprocal_lattice(test.phases['Stibnite'],
                                #tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)))

    # qmax=tth_2_q(tth_range[1], wavelength=test.wavelength)

    all_hkls = list(phase.lattice.get_allowed_hkl(qmax=qmax))
    all_qs = phase.Q(all_hkls)
    all_fs = np.abs(phase.StructureFactor(all_qs))**2
    rescale_array(all_fs, arr_min = 0, upper=100)

    return all_hkls, all_qs, all_fs

In [ ]:
from xrdmaptools.utilities.math import tth_2_q

phase = test.phases['stibnite']

all_hkls, all_qs, all_fs = generate_reciprocal_lattice(phase, qmax=tth_2_q(np.max(test.tth_arr), wavelength=test.wavelength))

In [ ]:
from xrdmaptools.geometry.geometry import nearest_q_on_ewald

q_near, q_dist = nearest_q_on_ewald(all_qs, test.wavelength, near_thresh=None)

In [ ]:
from xrdmaptools.geometry.geometry import nearest_polar_on_ewald

tth, chi = nearest_polar_on_ewald(all_qs, test.wavelength,
                            near_thresh=0.1, degrees=True)

In [ ]:
from xrdmaptools.geometry.geometry import nearest_pixels_on_ewald

img_coords = nearest_pixels_on_ewald(all_qs, test.wavelength, test.tth_arr, test.chi_arr,
                            near_thresh=0.05, degrees=True)

In [ ]:
from xrdmaptools.crystal.orientation import g_func

pixel_indices = 14, 29

plot_qs = all_qs @ orientation_map[pixel_indices].T
plot_qs = all_qs @ g_func(*euler_map[pixel_indices])

img_coords = nearest_pixels_on_ewald(plot_qs, test.wavelength, test.tth_arr, test.chi_arr,
                                near_thresh=0.2, degrees=True)

fig, ax = test.plot_image(test.map.images[pixel_indices], return_plot=True)

ax.scatter(*img_coords.T, s=5, c='r')

fig.show()

In [ ]:
test.plot_image(test.chi_arr)

In [ ]:
from xrdmaptools.crystal.orientation import g_func
g = g_func(*euler_map.reshape(test.map.num_images, 3).T)

ipole = [0, 1, 0]

colors, r, theta = [], [], []
for i in range(len(g)):
    
    # Take only part of the orientation matrix
    V = np.dot(g[i], ipole) # order is g, then ipole

    # Determine pixel color according to unit triangle
    R, G, B = 0, 0, 0
    base = 30 / 255 #all pixels are a bit brighter
    R=np.abs(V[2]) + base # referenced to crystallographic axes, but should be able to reference any axis
    G=np.abs(V[0]) + base
    B=np.abs(V[1]) + base

    # Normalize and brighten colors
    max_c = np.max([R,G,B])
    R = R / max_c
    G = G / max_c
    B = B / max_c
    
    # Setting pixel transparency if called
    A=1
    colors.append([R, G, B, A])

colors = np.asarray(colors)
colors = colors.reshape((*test.map.map_shape, 4))

fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=200)

ax.imshow(colors)
ax.set_title(str(ipole))

plt.show()

ValueError: cannot reshape array of size 119556 into shape (81,41,4)

In [ ]:
test.plot_interactive_map(display_map=colors, img_vmin=0, img_vmax=0.5)

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

phase = test.phases['stibnite']

#amp, x0, fwhm
p0 = [0]
fwhm = 0.1

for i in range(len(phase.reflections['hkl'])):
    p0.append(phase.reflections['int'][i]) # amp
    #p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(fwhm) #fwhm

print(p0)
calc_xrd = GaussianFunctions.multi_1d(test.tth, *p0)
calc_xrd = rescale_array(calc_xrd, 0, 1)

p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    #p0.append(phase.reflections['int'][i]) # amp
    p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(fwhm) #fwhm

print(p0)
xrd_mask = GaussianFunctions.multi_1d(test.tth, *p0)
xrd_mask = rescale_array(xrd_mask, 0, 1)

[0, 9.229291336271558, 9.79391082638766, 0.1, 36.35607901077193, 9.947146203273547, 0.1, 4.538280049707817, 10.003918314667498, 0.1, 100.0, 10.287225195166958, 0.1, 52.062489968299765, 11.203251751192864, 0.1, 1.658225681571489, 11.362641176238784, 0.1, 48.06295385493908, 11.393796968970216, 0.1, 9.900411012398704, 11.643708106935721, 0.1, 5.248101535050058, 11.661966917135357, 0.1, 1.1602521706592916, 12.582758827750855, 0.1, 2.2555150424646198, 12.672729437925394, 0.1, 25.048460183122817, 12.876186484052784, 0.1, 4.234661867419972, 12.977123761304796, 0.1, 52.24625259837442, 13.05898401587513, 0.1, 33.33036447295974, 13.28339128375106, 0.1, 1.5935045076294014, 13.397236314228078, 0.1, 41.25420645687469, 13.610946139998532, 0.1, 14.705669354514475, 13.652674016230323, 0.1, 23.458776258920437, 14.095268845670564, 0.1, 0.6725457075094272, 14.155715299043614, 0.1, 17.725053635792502, 14.681782907483074, 0.1, 35.33326441299243, 14.706014883021318, 0.1, 1.7257650652317706, 15.6492826713532

In [ ]:
integrations = plot_integrations.copy()
integrations = test.map.integrations.copy()
#integrations = integration_map.copy()

#rescale_array(integrations, arr_min = 0, upper=1)

proj_map = np.dot(integrations, calc_xrd)
proj_mask = np.dot(integrations, xrd_mask)

norms = np.linalg.norm(integrations, axis=-1)
norm_integrations = (integrations.T / norms.T).T

norm_calc_xrd = calc_xrd / np.linalg.norm(calc_xrd)
norm_xrd_mask = xrd_mask / np.linalg.norm(xrd_mask)

corr_map = np.dot(norm_integrations, norm_calc_xrd)
corr_mask = np.dot(norm_integrations, norm_xrd_mask)

window_int = integrations * xrd_mask
window_int /= np.linalg.norm(window_int)

double_map = np.dot(window_int, norm_calc_xrd)
double_mask = np.dot(window_int, norm_xrd_mask)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(7, 5), dpi=200)
axes = ax.ravel()

maps = [
        proj_map, corr_map, double_map,
        #proj_mask, corr_mask, double_mask
        ]
titles = [
          'Powder Projection',
          'Powder Correlation',
          'Powder Window Correlation',
         # 'Planes Projection',
         # 'Planes Correlation',
         # 'Planes Window Correlation'
          ]

fig.suptitle(f'{phase.name} mapping')


for axi, mapi, title in zip(axes, maps, titles):
    im = axi.imshow(mapi)
    fig.colorbar(im, ax=axi, shrink=0.7)
    axi.set_title(title)

fig.show()

In [ ]:
plt.close('all')

In [ ]:
#test.map.estimate_background(method='bruckner', binning=2, min_prominence=1)
test.map.estimate_background(method='bruckner', binning=2, min_prominence=0.05)

Estimating background with Bruckner algorithm.


  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:02<00:00, 42.73it/s]


In [ ]:
test.map.remove_background()

Removing background...done!


In [ ]:
test.map.integrations = test.map.integrations.reshape(*test.map.map_shape, -1)

In [ ]:
test.plot_image()

In [ ]:
from xrdmaptools.plot.interactive_plotting import interactive_dynamic_1d_plot

interactive_dynamic_1d_plot(rescale_array(integration_map, arr_min=0, upper=100), tth=test.tth, 
                                bkg_removal=None, ball_size=None, normalize=None,
                                display_map=None, display_title=None,
                                map_vmin=None, map_vmax=None,
                                cmap='viridis', marker_color='red')

In [ ]:
cif_dir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Literature\\CIF\\'''
test.clear_phases()
#test.load_phase('Barker\\1521772COD.cif', filedir=cif_dir, phase_name="liNbO3 old")
#test.load_phase('Barker\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 148")
#test.load_phase('Unknown\\LiNbO3.cif', filedir=cif_dir, phase_name="LiNbO3 new")
test.load_phase('AMCSD\\Platinum_0011157.cif', filedir=cif_dir, phase_name="platinum")
#test.load_phase('AMCSD\\Calcite_0000984.cif', filedir=cif_dir, phase_name="calcite")
test.load_phase('AMCSD\\Stibnite_0008636.cif', filedir=cif_dir, phase_name="stibnite")
#test.load_phase('AMCSD\\Silicon_0011243.cif', filedir=cif_dir, phase_name="silicon")
#test.load_phase('AMCSD\\Corundum_0009327.cif', filedir=cif_dir, phase_name="sapphire")
#test.load_phase('AMCSD\\Hematite_0017806.cif', filedir=cif_dir, phase_name="hematite")
#test.load_phase('AMCSD\\Iron-alpha_0011214.cif', filedir=cif_dir, phase_name="iron-ferrite")
#test.load_phase('AMCSD\\Iron-beta_0011215.cif', filedir=cif_dir, phase_name="iron-beta")
#test.load_phase('AMCSD\\Iron-delta_0011216.cif', filedir=cif_dir, phase_name="iron-delta")
#test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="iron-austenite")
#test.load_phase('AMCSD\\Cementite_0013523.cif', filedir=cif_dir, phase_name="cementite")
#test.load_phase('AMCSD\\Graphite_0011247.cif', filedir=cif_dir, phase_name="graphite")
#test.load_phase('AMCSD\\Tungsten_0011261.cif', filedir=cif_dir, phase_name="tungsten_1")
test.load_phase('AMCSD\\Tungsten_0011236.cif', filedir=cif_dir, phase_name="tungsten_2")
test.load_phase('AMCSD\\Iron_0011146.cif', filedir=cif_dir, phase_name="austenite")
test.phases['austenite'].a = 3.529

In [ ]:
[phase.get_hkl_reflections(tth_range=(np.min(test.tth), np.max(test.tth)),
                            ignore_less=0)
    for phase in test.phases.values()];

In [ ]:
from xrdmaptools.utilities.image_corrections import rescale_array

phase = test.phases['stibnite']

#amp, x0, fwhm
p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    p0.append(phase.reflections['int'][i]) # amp
    #p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(0.1) #fwhm

calc_xrd = GaussianFunctions.multi_1d(test.tth, *p0)
calc_xrd = rescale_array(calc_xrd, 0, 1)

p0 = [0]

for i in range(len(phase.reflections['hkl'])):
    #p0.append(phase.reflections['int'][i]) # amp
    p0.append(1) # amp
    p0.append(phase.reflections['tth'][i]) # x0
    p0.append(0.1) #fwhm

xrd_mask = GaussianFunctions.multi_1d(test.tth, *p0)
xrd_mask = rescale_array(calc_xrd, 0, 1)

In [ ]:
integrations = integration_map
#integrations = test.map.integrations

proj_map = np.dot(integrations, calc_xrd)

norms = np.linalg.norm(integrations, axis=-1)
norm_integrations = (integrations.T / norms.T).T

norm_calc_xrd = calc_xrd / np.linalg.norm(calc_xrd)

corr_map = np.dot(norm_integrations, norm_calc_xrd)

window_int = integrations * xrd_mask
window_int /= np.linalg.norm(window_int)

double_map = np.dot(window_int, norm_calc_xrd)

In [ ]:
test.plot_map(proj_map)

In [ ]:
test.plot_map(corr_map)

In [ ]:
test.plot_map(double_map)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(test.tth, rescale_array(calc_xrd, arr_min=0, upper=1))
ax.plot(test.tth, test.map.integrations[8, 160])
ax.plot(test.tth, rescale_array(calc_xrd, arr_min=0, upper=1) * rescale_array(test.map.integrations[8, 160], arr_min=0, upper=1))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.plot(tth, calc_xrd)

fig.show()

In [ ]:
test.initial_spot_analysis()

Extracting more information from peak parameters...


100%|██████████| 61352/61352 [00:45<00:00, 1354.13it/s]

Converting peaks positions to q-space...done!


In [ ]:
test.plot_image([6, 31], spots=True)

In [ ]:
new_phase_list = [phase_list[1], phase_list[0], phase_list[2], phase_list[3]]

In [ ]:
indices = [6, 31]
pixel_df = test.pixel_spots(indices)

spot_q_vals = np.linalg.norm(pixel_df[['qx', 'qy', 'qz']].values, axis=1)



In [ ]:
pixel_df

,map_x,map_y,guess_height,guess_img_x,guess_img_y,guess_tth,guess_chi,guess_cen_tth,guess_cen_chi,guess_fwhm_tth,guess_fwhm_chi,guess_int,guess_integrated,guess_fwhm_a,guess_fwhm_b,guess_rot_fwhm_tth,guess_rot_fwhm_chi,qx,qy,qz
19900,6,31,22.055635,173,1106,31.295430,12.541561,31.291277,12.545436,0.082449,0.229291,877.689575,877.689575,0.082449,0.229291,0.082449,0.229291,-1.259837,0.026379,-0.078638
19901,6,31,19.890331,188,1110,31.320581,11.948736,31.317982,11.950029,0.081982,0.200145,494.598877,494.598877,0.081982,0.200145,0.081982,0.200145,-0.808756,0.572924,-0.048576
19902,6,31,17.665329,164,1104,31.289520,12.894376,31.284570,12.874092,0.083074,0.219613,555.382080,555.382080,0.083074,0.219613,0.083074,0.219613,-1.265171,-0.402092,-0.087315
19903,6,31,10.836966,149,1101,31.288353,13.479495,31.286928,13.477103,0.079875,0.227869,311.315277,311.315277,0.079875,0.227869,0.079875,0.227869,-0.799469,-1.029969,-0.084213
19904,6,31,11.885958,844,283,15.995026,-25.331395,16.003747,-25.324092,0.080220,0.385215,340.319122,340.319122,0.080220,0.385215,0.080220,0.385215,-2.900456,0.561880,-19.830781
19905,6,31,7.239217,143,1099,31.272962,13.719568,31.273598,13.701554,0.082580,0.212817,256.482544,256.482544,0.082580,0.212817,0.082580,0.212817,-0.606662,-1.303437,-0.102486
19906,6,31,3.965852,202,1110,31.272779,11.419869,31.271974,11.430003,0.084827,0.240880,166.790741,166.790741,0.084827,0.240880,0.084827,0.240880,-0.611975,1.318935,-0.104835
19907,6,31,3.442822,279,545,19.952324,12.561981,19.951483,12.546825,0.075307,0.355158,123.857353,123.857353,0.075307,0.355158,0.075307,0.355158,9.039915,-0.176715,-5.555474
19908,6,31,3.839542,266,542,19.948836,13.328160,19.942979,13.317172,0.075162,0.329447,100.860748,100.860748,0.075162,0.329447,0.075162,0.329447,6.582010,6.141639,-5.478746
19909,6,31,2.936779,573,1413,36.893603,-2.549382,36.892723,-2.549681,0.083057,0.180251,77.162994,77.162994,0.083057,0.180251,0.083057,0.180251,6.071120,4.081783,-3.120611


In [ ]:
from xrdmaptools.geometry.geometry import get_q_vect

get_q_vect(16, -31, test.wavelength, degrees=True)

array([[ 2.39467839],
       [-1.43886794],
       [-0.39263059]])

In [ ]:
test.plot_image(test.q_arr[2])

In [ ]:
spot_q_vals

array([ 1.26256417,  0.99231431,  1.33039876,  1.30655192, 20.04964433,
        1.44135023,  1.45777025, 10.61200194, 10.53846494,  7.95346865,
        8.05444957, 20.08201186,  8.19029211,  8.12891677,  1.58015398,
        7.36000348, 17.00897523, 20.00164912])

In [ ]:
# Phase matching/mapping

#from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm

near_thresh = 0.05
sum_key = 'guess_int'
method = 'maximize_phase'
#method = 'maximize_spots'

phase_keys = list(test.phases.keys())
phase_list = list(test.phases.values())
#phase_list = new_phase_list

[phase.get_hkl_reflections(tth_range=(np.min(test.tth_arr), np.max(test.tth_arr)),
                               ignore_less=1)
        for phase in phase_list];

plot_map = np.zeros(test.map.map_shape)
plot_map[:] = 0

plot_dict = {}
for phase in phase_list:
    plot_dict[phase.name] = plot_map.copy()
plot_dict['unmatched'] = plot_map.copy()

for index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(index, test.map.map_shape)

    image = test.map.images[indices]
    pixel_df = test.pixel_spots(indices)
    #pixel_df = pixel_df.copy().dropna()
    num_spots = len(pixel_df)
    
    if num_spots > 1:
        spot_q_vals = np.linalg.norm(pixel_df[['qx', 'qy', 'qz']].values, axis=1)

        phase_diffs = []
        pot_matches = []
        phase_matches = []
        spot_matches = []
        for phase in phase_list:
            phase_q_vals = phase.reflections['q']
            #dist_arr = euclidean_distances(phase_q_vals.reshape(-1, 1), spot_q_vals.reshape(-1, 1))
            diff_arr = np.abs(phase_q_vals[:, np.newaxis] - spot_q_vals[np.newaxis, :])
            phase_diffs.append(diff_arr)
            pot_match = diff_arr < near_thresh
            pot_matches.append(pot_match)

            # Fraction of phase reflections explained by found spots
            #phase_matches.append(np.sum(pot_match / len(test.phases)))
            phase_matches.append(np.sum(np.any(pot_matches[-1], axis=1)) / len(phase_q_vals))

            # Number of spots explained by a given phase
            spot_matches.append(np.sum(np.any(pot_matches[-1], axis=0)))
            #raise

        # Track which spots have been matched
        rem_spot_mask = [True,] * num_spots

        if method in ['maximize_phase']:
            # Maximize phase matches
            # Should work better for polycrystalline patterns
            while (np.any(rem_spot_mask)) and (np.sum(phase_matches) > 0):
                # Check for more than 1 fully represented array
                if np.sum(np.array(phase_matches) == 1) > 1:
                    stacked_diffs = np.stack([np.min(diffs, axis=0) for diffs in phase_diffs])
                    spot_mask = np.min(stacked_diffs, axis=0) < near_thresh
                    phase_ind = np.argmin(stacked_diffs, axis=0)
                    phase_ind[~spot_mask] = -1
                    phase_mask = (np.array(phase_matches) == 1)
                    phase_count = [0,] * len(phase_list)
                    for ind in phase_ind:
                        if phase_mask[ind]:
                            phase_count[ind] += 1
                    max_ind = np.argmax(phase_count)
                else:
                    max_ind = np.argmax(phase_matches)

                # Index matching spots
                spot_mask = np.any(pot_matches[max_ind], axis=0)
                # Remove indexed spots from future use
                spot_mask *= rem_spot_mask
                rem_spot_mask *= ~spot_mask

                # Finally, find the value and remove reflections
                val = np.nansum(pixel_df[sum_key].values[spot_mask])
                phase_matches[max_ind] = 0

                # Fill value
                plot_dict[phase_list[max_ind].name][indices] = val

        elif method in ['maximize_spots']:
            # Maxmize spot_matches
            # Not sure when this one should work better
            while (np.any(rem_spot_mask)) and (np.sum(spot_matches) > 0):
                max_ind = np.argmax(spot_matches)
                # Index matching spots
                spot_mask = np.any(pot_matches[max_ind], axis=0)
                # Remove indexed spots from future use
                spot_mask *= rem_spot_mask
                rem_spot_mask *= ~spot_mask

                # Finally, find the value
                val = np.nansum(pixel_df[sum_key].values[spot_mask])
                spot_matches[max_ind] = 0

                # Fill value
                plot_dict[phase_list[max_ind].name][indices] = val

        elif method in ['minimize_q_error']:
        # Minimize Q difference
        # Should work better for perfect crystals and detector calibration...
            stacked_diffs = np.stack([np.min(diffs, axis=0) for diffs in phase_diffs])
            spot_mask = np.min(stacked_diffs, axis=0) < near_thresh
            phase_ind = np.argmin(stacked_diffs, axis=0)
            phase_ind[~spot_mask] = -1
            for i in range(len(phase_list)):
                val = np.nansum(pixel_df[sum_key].values[phase_ind == i])
                plot_dict[phase_list[i].name][indices] = val
            rem_spot_mask = ~spot_mask

        # Collect unmatched spots
        if np.any(rem_spot_mask):
            val = np.nansum(pixel_df[sum_key].values[rem_spot_mask])
            plot_dict['unmatched'][indices] = val


100%|██████████| 3381/3381 [00:01<00:00, 2290.71it/s]


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(5, 5), dpi=200)
ax = axes.ravel()

for i, key in enumerate(plot_dict.keys()):
    if key == 'unmatched':
        pass
        continue

    im = ax[i].imshow(plot_dict[key], extent=test.map_extent())
    fig.colorbar(im, ax=ax[i], shrink=0.4)

    ax[i].set_title(key)

plt.tight_layout()

In [ ]:
plot_map = np.zeros(test.map.map_shape)
plot_map[:] = np.nan

for image_index in tqdm(range(test.map.num_images)):
    indices = np.unravel_index(image_index, test.map.map_shape)
    pixel_df = test.spots[(test.spots['map_x'] == indices[0])
                          & (test.spots['map_y'] == indices[1])]
    
    if len(pixel_df) < 1:
        plot_map[indices] = 0
    else:
        #plot_map[indices] = np.sum(pixel_df['guess_int'])
        #plot_map[indices] = np.sum(pixel_df['fit_amp'])
        #plot_map[indices] = np.sum(pixel_df['fit_integrated'])
        #plot_map[indices] = np.sum(pixel_df['fit_fwhm_chi'])
        #plot_map[indices] = np.mean(pixel_df['fit_chi0'])
        #plot_map[indices] = len(pixel_df)
        #plot_map[indices] = np.nanmedian(pixel_df['qz'])
        #plot_map[indices] = np.nanmin(pixel_df['fit_r_squared'])
        plot_map[indices] = pixel_df.iloc[np.argmax(pixel_df['guess_int'])]['guess_fwhm_chi']
        #plot_map[indices] = pixel_df.iloc[np.argmax(pixel_df['fit_integrated'])]['fit_chi0']


fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

im = ax.imshow(plot_map)
fig.colorbar(im, ax=ax)
ax.set_title('Spot Value')

plt.show()

100%|██████████| 19656/19656 [00:29<00:00, 664.19it/s]


In [ ]:
def derivative2(input, axis, output, mode, cval):
    #return correlate1d(input, [1, -2, 1], axis, output, mode, cval, 0)
    return correlate1d(input, [1, 1, -8, 1, 1], axis, output, mode, cval, 0)

In [ ]:
lap_image = generic_laplace(sob_image, derivative2)

In [ ]:
test.plot_image(-lap_image)

In [ ]:
test_image = test.map.images[8, 34].copy()
mask = test.map.blob_masks[8, 34]

In [ ]:
sob_image = test_image.copy()
sob_image = gaussian_filter(sob_image, sigma=1)
#sob_x = sobel(sob_image, axis=1)
#sob_y = sobel(sob_image, axis=0)
sob_x = prewitt(sob_image, axis=1)
sob_y = prewitt(sob_image, axis=0)
sob_mag = np.sqrt(sob_x**2 + sob_y**2)
div = sobel(sob_x, axis=1) + sobel(sob_y, axis=0)
mask = div > 2

In [ ]:
test.plot_image(gaussian_filter(test_image, sigma=0) - gaussian_filter(test_image, sigma=1))

In [ ]:
extent = 10
max_h, max_v = np.unravel_index(np.argmax(-div), div.shape)
h_bounds = (max_v - extent, max_v + extent)
v_bounds = (max_h - extent, max_h + extent)
bound_image = sob_image[v_bounds[0] : v_bounds[1], h_bounds[0] : h_bounds[1]]

In [ ]:
test.plot_image(sob_x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=200)

ax.imshow(bound_image)

scale = 1

sob_x = sobel(bound_image, axis=1) * scale
sob_y = sobel(bound_image, axis=0) * scale

for index in range(np.prod([*bound_image.shape])):
    indices = np.unravel_index(index, bound_image.shape)

    dx = sob_y[indices]
    dy = sob_x[indices]
    

    #if indices[0] > bound_image.shape[0] // 2 - 1:
    #    dx = -dx
    #if indices[1] > bound_image.shape[1] // 2 - 1:
    #    dy = -dy

    ax.arrow(*indices, dx, dy, width=0.00001, head_width=0.1, fc='r', ec='r')


plt.show()